In [5]:
import os
import PyPDF2
import pdfplumber
import fitz  # PyMuPDF
#from pdfminer.pdfparser import PDFParser
#from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage

directory = os.getcwd()  # Замените на путь к вашей директории
pdf_files = []
Number_of_pages = []

# Обход директории и вложенных папок
for root, dirs, files in os.walk(directory):
    for file in files:
        if file.endswith(".pdf"):
            # Добавляем полный путь к файлу в список
#            pdf_files.append(os.path.join(root, file))
            pdf_files.append(file)
fil = open('inform_about_pdf.txt', 'w')

for filename in pdf_files:
    print()
    print('filename = ', filename)
    fil.write('\n')
    fil.write('filename = ' + filename + '\n')

    try:
        with open(filename, "rb") as file:
            reader = PyPDF2.PdfReader(file)
            number_of_pages = len(reader.pages)
            print( f"PyPDF2.PdfReader(file) Количество страниц в документе: {number_of_pages}")
    except:
        number_of_pages = - 100

    fil.write(f"PyPDF2.PdfReader(file) Количество страниц в документе: {number_of_pages}\n")

    try:
        with pdfplumber.open(filename) as pdf:
            number_of_pages1 = len(pdf.pages)
            print('pdfplumber.open(filename)', f"Количество страниц в документе: {number_of_pages1}")
    except:
        number_of_pages1 = - 100
    fil.write(f"pdfplumber.open(filename) Количество страниц в документе: {number_of_pages1}\n")

    try:
        doc = fitz.open(filename)
        number_of_pages2 = doc.page_count
        print('fitz.open(filename)', f"Количество страниц в документе: {number_of_pages2}")
        doc.close()
    except:
        number_of_pages2 = - 100
    fil.write(f"fitz.open(filename) Количество страниц в документе: {number_of_pages2}\n")

    try:
        with open(filename, "rb") as file:
            # Инициализируем подсчёт страниц
            number_of_pages3 = 0
            # Проходим по страницам PDF
            for page in PDFPage.get_pages(file):
                number_of_pages3 += 1
        print('PDFPage.get_pages(file)', f"Количество страниц: {number_of_pages3}")
    except:
        number_of_pages3 = - 100

    fil.write(f"PDFPage.get_pages(file) Количество страниц: {number_of_pages3}\n")
    Number_of_pages.append((number_of_pages, number_of_pages1, number_of_pages2, number_of_pages3))
fil.close()


filename =  Крайко А.Н. Краткий курс теоретической газовой динамики. 2007.pdf
PyPDF2.PdfReader(file) Количество страниц в документе: 302
pdfplumber.open(filename) Количество страниц в документе: 302
fitz.open(filename) Количество страниц в документе: 302
PDFPage.get_pages(file) Количество страниц: 302


In [6]:
Number_of_pages

[(302, 302, 302, 302)]

In [7]:
import pandas as pd
from PyPDF2 import PdfReader
import fitz  # PyMuPDF
import pdfplumber
from pdfminer.high_level import extract_pages
from PyPDF4 import PdfFileReader



def count_pages_all_methods(pdf_path):
    results = []

    # Метод 1: PyPDF2
    try:
        reader = PdfReader(pdf_path)
        results.append(len(reader.pages))
    except Exception as e:
        print(f"Ошибка в PyPDF2 ({pdf_path}): {e}")
        results.append(None)

    # Метод 2: PyMuPDF (fitz)
    try:
        doc = fitz.open(pdf_path)
        results.append(doc.page_count)
    except Exception as e:
        print(f"Ошибка в PyMuPDF ({pdf_path}): {e}")
        results.append(None)

    # Метод 3: pdfplumber
    try:
        with pdfplumber.open(pdf_path) as pdf:
            results.append(len(pdf.pages))
    except Exception as e:
        print(f"Ошибка в pdfplumber ({pdf_path}): {e}")
        results.append(None)

    # Метод 4: PDFMiner
    try:
        print(1/0)
        num_pages = sum(1 for _ in extract_pages(pdf_path))
        results.append(num_pages)
    except Exception as e:
        print(f"Ошибка в PDFMiner ({pdf_path}): {e}")
        results.append(None)

    # Метод 5: PyPDF4
    try:
        with open(pdf_path, 'rb') as f:
            reader = PdfFileReader(f)
            results.append(reader.numPages)
    except Exception as e:
        print(f"Ошибка в PyPDF4 ({pdf_path}): {e}")
        results.append(None)

    return tuple(results)


# Список PDF файлов

directory = os.getcwd()  # Замените на путь к вашей директории
pdf_files = []
Number_of_pages = []

# Обход директории и вложенных папок
for root, dirs, files in os.walk(directory):
    for file in files:
        if file.endswith(".pdf"):
            # Добавляем полный путь к файлу в список
#            pdf_files.append(os.path.join(root, file))
            pdf_files.append(file)




# Создание общего списка
results_list = []
for pdf_file in pdf_files:
    page_counts = count_pages_all_methods(pdf_file)
    results_list.append((pdf_file, *page_counts))

# Создание DataFrame
columns = ["File Name", "PyPDF2", "PyMuPDF", "pdfplumber", "PDFMiner", "PyPDF4"]
df = pd.DataFrame(results_list, columns=columns)

# Сохранение DataFrame в CSV (если нужно)
df.to_csv("pdf_page_counts.csv", index=False, encoding = 'utf-8-sig')

# Вывод DataFrame
print(df)


Ошибка в PDFMiner (Крайко А.Н. Краткий курс теоретической газовой динамики. 2007.pdf): division by zero


                                           File Name  PyPDF2  PyMuPDF  \
0  Крайко А.Н. Краткий курс теоретической газовой...     302      302   

   pdfplumber PDFMiner  PyPDF4  
0         302     None     302  


In [8]:
import pandas as pd
from PyPDF2 import PdfReader
import fitz  # PyMuPDF
import pdfplumber
from pdfminer.high_level import extract_pages
from PyPDF4 import PdfFileReader
import time
import os


def count_pages_all_methods(pdf_path):
    results = []

    t0 = time.time()
    # Метод 1: PyPDF2
    try:
        reader = PdfReader(pdf_path)
        l = len(reader.pages)
    except Exception as e:
        print(f"Ошибка в PyPDF2 ({pdf_path}): {e}")
        l = None

    t1 = time.time()
    results.append((l, f'{t1 - t0:.2f}'))

    # Метод 2: PyMuPDF (fitz)
    try:
        doc = fitz.open(pdf_path)
        l = doc.page_count
    except Exception as e:
        print(f"Ошибка в PyMuPDF ({pdf_path}): {e}")
        l = None

    t2 = time.time()
    results.append((l, f'{t2 - t1:.2f}'))
    # Метод 3: pdfplumber
    try:
        with pdfplumber.open(pdf_path) as pdf:
            l = len(pdf.pages)
    except Exception as e:
        print(f"Ошибка в pdfplumber ({pdf_path}): {e}")
        l = None

    t3 = time.time()
    results.append((l, f'{t3 - t2:.2f}'))
    # Метод 4: PDFMiner
    try:
        print(1/0)
        num_pages = sum(1 for _ in extract_pages(pdf_path))
        l = num_pages
    except Exception as e:
        print(f"Ошибка в PDFMiner ({pdf_path}): {e}")
        l = None

    t4 = time.time()
    results.append((l, f'{t4 - t3:.2f}'))
    # Метод 5: PyPDF4
    try:
        with open(pdf_path, 'rb') as f:
            reader = PdfFileReader(f)
            l = reader.numPages
    except Exception as e:
        print(f"Ошибка в PyPDF4 ({pdf_path}): {e}")
        l = None
    t5 = time.time()
    results.append((l, f'{t5 - t4:.2f}'))
    return tuple(results)


# Список PDF файлов

directory = os.getcwd()  # Замените на путь к вашей директории
pdf_files = []
Number_of_pages = []

# Обход директории и вложенных папок
for root, dirs, files in os.walk(directory):
    for file in files:
        if file.endswith(".pdf"):
            # Добавляем полный путь к файлу в список
#            pdf_files.append(os.path.join(root, file))
            pdf_files.append(file)




# Создание общего списка
results_list = []
for pdf_file in pdf_files:
    page_counts = count_pages_all_methods(pdf_file)
    results_list.append((pdf_file, *page_counts))

# Создание DataFrame
columns = ["File Name", "PyPDF2", "PyMuPDF", "pdfplumber", "PDFMiner", "PyPDF4"]
df = pd.DataFrame(results_list, columns=columns)

# Сохранение DataFrame в CSV (если нужно)
try:
    df.to_csv("pdf_page_counts.csv", index=False, encoding = 'utf-8-sig')
except:
    df.to_csv("pdf_page_counts1.csv", index=False, encoding = 'utf-8-sig')

# Вывод DataFrame
print(df)


Ошибка в PDFMiner (Крайко А.Н. Краткий курс теоретической газовой динамики. 2007.pdf): division by zero


                                           File Name       PyPDF2  \
0  Крайко А.Н. Краткий курс теоретической газовой...  (302, 0.59)   

       PyMuPDF   pdfplumber      PDFMiner       PyPDF4  
0  (302, 0.01)  (302, 1.05)  (None, 0.00)  (302, 0.30)  
